In [68]:
# import libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline            

### Read data from file and create vectors
The function 'get_vectors' takes a filename as input and returns a matrix X containing each row was a datapoint and vector y containing target labels.

In [69]:
def get_vectors(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    labels = []
    
    with f:
        line = f.readline()
        while line != '':
            # strip newline and outer parenthesis
            line = line.strip('\n')
            line = line.strip('( )')
            
            # extrace label and append to labels list
            single_label = line.split('), ')[-1]
            labels.append(single_label)
            
            # extrace features and append to features list
            feat = line.split('), ')[0].split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # create dataframe of features and append labels
        X = np.array(features, dtype = float, ndmin = 2)
        
        # convert labels list to array
        y = np.array(labels, dtype = str, ndmin = 2)
        
        return X, y.transpose()

### Distance Calculation

In [70]:
# This function calculates euclidean distance between training datapoints and test data point
def get_euclidean_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_squared_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_squared_diff = sum_of_squared_diff + (x_vector - p[i])**2
        
    # take sq root to get array of cartesianeuclidean distance
    euc_dist = np.sqrt(sum_of_squared_diff)
    
    return euc_dist

# This function calculates manhattan distance between training datapoints and test data point
def get_manhattan_distance(X_train, p):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    sum_of_abs_diff = np.zeros((n, 1), dtype = float)
    
    # use vectorization to get sum of squared difference
    for i in range(f_n):
        x_vector = X_train[:,i].reshape((n,1))
        sum_of_abs_diff = sum_of_abs_diff + abs(x_vector - p[i])
        
    # take sq root to get array of cartesianeuclidean distance
    man_dist = np.sqrt(sum_of_abs_diff)
    
    return man_dist


### Leave-one-out evaluation function 

This function takes the entire feature dataset(X) , correct data point labels(y), the number of neighbors to consider(k) and the type of distance to use for nearest neighbor calculation(dist_type) as arguments and returns the accuracy (total correct predictions/ total datapoints). 

The datapoint ot be left out and tested is chosen according to its index (i.e. item at index 0 is left out during the first iteration and item at index n-1 is left our during the last iteration).

In [71]:
def leave_one_out_evaluation(X, y, k, dist_type):
    
    # get number of training items and number of features
    n, f_n = X.shape
    
    # prediction labels generated by 'predict_class_with_knn' will be stored in this list
    predictions = []
    
    for i in range(n):
        X_train = np.delete(X, i, axis = 0)
        y_train = np.delete(y, i, axis = 0)
        X_test = X[i,:]
        pred, prob = predict_class_with_knn(X_train, y_train, X_test, k, dist_type)
        predictions.append(pred)
    
    # convert prediction list to numpy array
    predictions = np.array(predictions, dtype = str, ndmin = 2)
    predictions = predictions.reshape(predictions.shape[1], 1)
    
    # return accuracy
    return (np.sum(y == predictions))/n

## 2 b)

### KNN Implementation 

This function takes training data, prediction datapoint and number of neighbors as the argument and returns
a prediction class based on highest MAP value i.e. argmax_c P(class|<data>)

Key notes:
1. Cartesian/euclidean distance is used as a default distance measure.
2. The distance and labels of the first 'k' neighbors are printed.
3. MAP values of each class is calculated and stored in a dictionary with class labels as the key.
4. Tie breaker: If two or more classes have the same highest probability value, the first occuring class is returned.

In [72]:

def predict_class_with_knn(X_train, y_train, p, k, dist_type):
    
    # n = total number of datapoints, f_n = total number of features
    n, f_n = X_train.shape
    
    if dist_type == 'euclidean':
        dist_arr = get_euclidean_distance(X_train, p)
    elif dist_type == 'manhattan':
        dist_arr = get_manhattan_distance(X_train, p)
    
    # concat with y_train labels and sort in ascending order of the distance
    dist_arr = np.concatenate((dist_arr, y_train), axis = 1)
    dist_arr = dist_arr[dist_arr[:,0].argsort()]
    
    # the first 'k' rows contain distance and labels of the k nearest neighbors
    knn = dist_arr[0:k,:]
    
    # save labels of the k nearest neighbors as a list s.t. to count occurence
    knn_labels = list(knn[:,1])
    
    # class labels
    class_labels = list(set(knn_labels))
    
    # calculate class probability
    class_probability = {}
    class_probability['Metal'] = knn_labels.count('Metal')/n
    class_probability['Ceramic'] = knn_labels.count('Ceramic')/n
    class_probability['Plastic'] = knn_labels.count('Plastic')/n
    
    '''
    The knn_label_prob list will store class probability values for each of the k nearest neighbors w.r.t their class 
    label so that if a tie occurs between two classes, class label of the neighbor that is closest to the test datapoint
    will be chosen.
    '''

    knn_label_prob = []
    for idx, item in enumerate(knn_labels):
        knn_label_prob.append(class_probability[item])
    
    
    max_class_prob_idx = 0
    max_class_prob = float(0)
    
    for i in range(k):
        if knn_label_prob[i] > max_class_prob:
            max_class_prob = knn_label_prob[i]
            max_class_prob_idx = i
    
    # return the class with maximum MLP and the probablity value
    return knn_labels[max_class_prob_idx], knn_label_prob[max_class_prob_idx]


### Provide training filename & prediction data point
The file must contain 1 datapoint per line in format (( height, diameter, weight, hue ), label ) which is similar to the format provided for the assignment

In [73]:
# provide training and test filename
fname_train = str(input('Enter file containing training data: '))
fname_test = str(input('Enter file containing test data: '))

# provide 'k' value
k = int(input('Enter value of k: '))
print('\n')

# get training data as vectors
X, y = get_vectors(fname_train)

# read test file to make predictions
with open(fname_test, 'r') as f:
    line = f.readline()
    i = 1
    while line != '':
        line = line.strip('\n')
        line = line.strip('( )')
        values = line.split(', ')
        p = np.array(values, dtype = float)
        preidicted_class, prob_value = predict_class_with_knn(X, y, p, k, 'euclidean')
        print(f"Datapoint {i} : '{preidicted_class}' with a probability of {prob_value:.4f}.")
        i = i + 1
        line = f.readline()
    

Enter file containing training data: Data\\2_a_train.txt
Enter file containing test data: Data\\2_a_test.txt
Enter value of k: 5


Datapoint 1 : 'Plastic' with a probability of 0.2500.
Datapoint 2 : 'Ceramic' with a probability of 0.1667.
Datapoint 3 : 'Plastic' with a probability of 0.2500.
Datapoint 4 : 'Plastic' with a probability of 0.2500.


## 2 c)

### Enter filename for leave-one-out evaluation

In [82]:
fname_loo = str(input('Enter filename for leave-one-out evaluation data: '))

X, y = get_vectors(fname_loo)

Enter filename for leave-one-out evaluation data: Data\\2_c_d_e.txt


### K = 1, 3 and 5 using cartesian distance

In [83]:
# initialize distionary to store accuracy values for different 'k' values
accuracy_euc = {}

# calculate accuracy for various values of 'k'
accuracy_euc[1] = leave_one_out_evaluation(X, y, 1, dist_type = 'euclidean')
accuracy_euc[3] = leave_one_out_evaluation(X, y, 3, dist_type = 'euclidean')
accuracy_euc[5] = leave_one_out_evaluation(X, y, 5, dist_type = 'euclidean')

print('With cartesian distance:')
for key, value in accuracy_euc.items():
    print(f'For k = {key} the accuracy is {value:0.6f}.')

With cartesian distance:
For k = 1 the accuracy is 0.425000.
For k = 3 the accuracy is 0.408333.
For k = 5 the accuracy is 0.400000.


### Conclusion: k = 1 was found to give the best performance in terms of accuracy.

## 2 d) 

### K = 1, 3 and 5 using manhattan distance

In [85]:
# initialize distionary to store accuracy values for different 'k' values
accuracy_man = {}

# calculate accuracy for various values of 'k'
accuracy_man[1] = leave_one_out_evaluation(X, y, 1, dist_type = 'manhattan')
accuracy_man[3] = leave_one_out_evaluation(X, y, 3, dist_type = 'manhattan')
accuracy_man[5] = leave_one_out_evaluation(X, y, 5, dist_type = 'manhattan')

print('With manhattan distance:')
for key, value in accuracy_man.items():
    print(f'For k = {key} the accuracy is {value:0.6f}.')

With manhattan distance:
For k = 1 the accuracy is 0.500000.
For k = 3 the accuracy is 0.458333.
For k = 5 the accuracy is 0.516667.


### Conclusion: Manhattan distance is found to perform better than cartesian distance. Accuracy is highest for a value of k = 5.

## 2 e) 

### Prediction after removing 4th attribute (hue) 

In [86]:
# remove 4th attribute from X
X = np.delete(X, 3, axis = 1)

In [87]:
# check shape of x
X.shape

(120, 3)

In [88]:
# initialize distionary to store accuracy values for different 'k' values
accuracy_euc3= {}

# calculate accuracy for various values of 'k'
accuracy_euc3[1] = leave_one_out_evaluation(X, y, 1, dist_type = 'euclidean')
accuracy_euc3[3] = leave_one_out_evaluation(X, y, 3, dist_type = 'euclidean')
accuracy_euc3[5] = leave_one_out_evaluation(X, y, 5, dist_type = 'euclidean')

print('With cartesian distance after removing 4th attribute:')
for key, value in accuracy_euc3.items():
    print(f'For k = {key} the accuracy is {value:0.6f}.')

With cartesian distance after removing 4th attribute:
For k = 1 the accuracy is 0.816667.
For k = 3 the accuracy is 0.833333.
For k = 5 the accuracy is 0.808333.


### Conclusion: Removing the 4th attribute was found to significantly improve accuracy with highest accuracy value for k = 3